# Feature Engineering

### Main flavors of data and feature engineering
* Tabular: Dataframe model
    * "Typical" business data tables
* Batch/Tensor/Vector: Array model
    * Numeric data, timeseries, scientific data, audio, images, video, geodata, etc.
* Natural language
    * Batches of strings
    * Transformed into array data through NLP-specific techniques
    
<img src='images/flow-transform.png' width=800>

### "Must-haves" for feature engineering on large data

* Some data representation for the large dataset
    * Likely distributed, out-of-core, lazy, streaming, etc.
* Mechanism to load data from standard formats and locations into the representation
    * E.g., loading HDF5 in S3 or Parquet in HDFS
* APIs to apply feature engineering transformations
    * Mathematical operations
    * String, date, etc.
    * Custom ("user-defined")
* Integration to a modeling framework and/or ability to write to standard formats

### "Nice-to-haves"

* Intuitive data representation: similar to "small data" tooling
* APIs that resemble those of the most common industry-standard libraries
* Both modeling integration *and* ability to write out transformed data

<img src='images/psf-logo@2x.png'/>

## Rise of Python

Python has become the *lingua franca* or dominant cross-cutting language for data science.

>
> __Note__ this is not to imply Python is the best or only language, or that other languages might not be intrinsically better or even, in the future, more successful. 
>
> There are wonderful things to be said for languages from Rust to R to Julia to many others, but for baseline data science capability and versatility in commercial enterprises today, it's Python
>

So we can turn to Python and look at the dominant libraries and tools within that ecosystem
* Tabular data: Pandas
* Array data: NumPy and derivatives like CuPy, JAX.numpy, etc.
* Basic modeling: scikit-learn, XGBoost, etc.
* Deep learning: PyTorch, Tensorflow
* NLP: SpaCy, NLTK, Huggingface, etc.

As we get into further parts of the workflow, like hyperparameter tuning or reinforcement learning there are more choices. 

For time reasons, we're going to stick to this core workflow of extraction through modeling and tuning, and not continue on into MLOps and deploment architectures, or meta-modeling platforms for experimentation, feature and provenance tracking, etc. That would be a bit too much to take on!

__Bottom line__: We want a data representation and APIs that are fairly close to the Pandas / NumPy / scikit-learn (SciPy) workflow. And we want elegant bridges into things like PyTorch, XGBoost, NLP tools, and tuning tools.

## Dask: SciPy at Scale

Luckily, Dask is well placed to solve this problem. 

While enterprises were still wrestling with JVM-based tools over the past 5 years, scientists, researchers, and others in the PyData and SciPy communities were building Dask, a pure-Python distributed compute platform that integrates deeply with all of the standard SciPy tools.

__What does this mean?__

We can take many of our local workflows to large-scale data via Dask with fairly minimal effort -- because under the hood, Dask is designed to use those "small data" structures in federation to create arbitrarily large ones.

<table align=left><tr><td>
    <img src='images/dask-dataframe.svg' width=350>
</td><td style='width:10em;'>
</td><td>
    <img src='images/dask-array.svg'></td></tr></table>

As an added bonus, due to the Dask architecture, it can leverage GPU-enabled versions of the underlying libraries.
* GPU + NumPy => CuPy
* GPU + Pandas => cuDF (RAPIDS CUDA dataframe)
* GPU + scikit-learn => cuML (RAPIDS CUDA algorithms)
etc.

### Using Dask for Feature Transformation

* We need to be able to load data in a standard format
* Manipulate it using dataframe or array APIs
* Write it and/or pass it efficiently to a modeling framework

In [ ]:
from dask import dataframe as ddf
from dask.distributed import Client

client = Client(n_workers=2, threads_per_worker=1, memory_limit='1GB')

client

In [ ]:
df = ddf.read_csv('data/diamonds.csv')
df

In [ ]:
df.head()

In [ ]:
df = df.drop(columns=['Unnamed: 0'])
df = df.categorize()

df

In [ ]:
prepared = ddf.reshape.get_dummies(df)

prepared.head()

# Modeling

<img src='images/flow-model.png' width=800/>

If Dask makes an easy choice for some feature engineering and preprocessing, we're back into the deep end making choices for modeling.

Why?

Simply put, different kinds of modeling are handled best by different tools, so we have a lot of choices to make.

* "Classic" ML
    * Dask
    * Dask ML
    * XGBoost (with or without Dask)
* Unsupervised learning and dimensionality reduction
    * Dask supports some algorithms
    * For others, we may want to scale a deep-learning tool (PyTorch/Tensorflow)
        * Horovod
        * Ray SGD
* Deep learning (scaling PyTorch/TF easily)
    * Horovod
    * Ray SGD
    * Ray RLlib for deep reinforcement learning
* Simulations and agent-based models
    * Ray for stateful-agent simulations
    * Dask Actors may be an option


## Example: Linear Model with Dask

In [ ]:
y = prepared.price.to_dask_array(lengths=True)
arr = prepared.drop('price', axis=1).to_dask_array(lengths=True)

arr[:4]

In [ ]:
arr[:4].compute()

In [ ]:
from dask_ml.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(arr, y, test_size=0.1)

X_train

In [ ]:
from dask_ml.linear_model import LinearRegression

lr = LinearRegression(solver='lbfgs', max_iter=10)
lr_model = lr.fit(X_train, y_train)

In [ ]:
y_predicted = lr_model.predict(X_test)

y_predicted

In [ ]:
from dask_ml.metrics import mean_squared_error
from math import sqrt

sqrt(mean_squared_error(y_test, y_predicted))

In [ ]:
client.close()

## What is Ray?

Ray (https://ray.io/) is a scale-out computing system designed for high-throughput, resilient stateful-actor algorithms. Ray was design at UC Berkeley's RISE lab under the supervision of some of the same team that created Apache Spark. 

Ray supports a number of languages at the API layer (Python and Java today) while most of the engine is C++. Ray's stateful actor support makes it strong in a number of key areas, like distributed SGD and reinforcement learning.

Let's try a reinforcement learning example!

> __Reinforcement Learning__ is a family of techniques that train *agents* to act in an *environment* to maximize *reward*. Famous examples include agents that can play chess, go, or Atari games ... but the field is hot because those agents can also be robots learning to do work, autonomous vehicles driving, or even virtual salesmen learning to get the best price possible from a customer.

Ray treats deep reinforcement learning (RL + deep learning) as a top-level use case and includes libraries that encapsulate many of the most popular algorithms.

Here, to create a simple example, we'll use __Deep Q-Learning__ (a foundational deep RL algorithm) to learn OpenAI's "cart-pole" (https://gym.openai.com/envs/CartPole-v1/) environment, which you can visualize like this:

<video src='images/cpv1.mp4' controls="true">

This example, and the lab, are based on the demo in Dean Wampler's excellent intro paper, "What is Ray?" on O'Reilly Safari Online: https://learning.oreilly.com/library/view/what-is-ray/9781492085768/

In [ ]:
import ray
import ray.rllib.agents.dqn as dqn

ray.shutdown()
ray.init()

In [ ]:
# Specifies the OpenAI Gym environment for CartPole, V1.
SELECT_ENV = "CartPole-v1"

# Number of training runs.
N_ITER = 50

# default configuration.
config = dqn.DEFAULT_CONFIG.copy()

# Suppress too many messages.
config["log_level"] = "WARN"

# Use > 1 for more CPU cores, e.g., over a cluster.
config['num_workers'] = 2

# Describe network
config['model']['fcnet_hiddens'] = [40,20]

# Don't pin a CPU core to each worker (allows more workers).
config['num_cpus_per_worker'] = 0
checkpoint_dir = 'checkpoints'

In [ ]:
trainer = dqn.DQNTrainer(config, SELECT_ENV)

In [ ]:
fmt = '{:3d},{:8.4f},{:8.4f},{:8.4f}'
last_checkpoint = ''
for n in range(N_ITER):
    result = trainer.train()
    min  = result['episode_reward_min']
    mean = result['episode_reward_mean']
    max  = result['episode_reward_max']
    last_checkpoint = trainer.save(checkpoint_dir)
    print(fmt.format(n, min, mean, max))
print(f'last checkpoint file: {last_checkpoint}')

__Note__: If you've worked with RL and OpenAI gym before, you may realize these are not particularly impressive numbers, and not a particularly impressive algorithm.

Don't worry: __Ray RLlib__ includes a variety of much more powerful algorithms which achieve better results. We'll try one of them -- Proximal Policy Optimization (PPO) in the lab exercise.

In [ ]:
ray.shutdown()